# James Jones
### 11-18-2022

In [1]:
# Basic imports and inspecting 'Data' folder
import os, time, json
import tmdbsimple as tmdb 
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm_notebook
FOLDER = "Data/"
os.makedirs(FOLDER, exist_ok=True)
os.listdir(FOLDER)

['tmdb_api_results_2000.json',
 ' final_tmdb_data_2000.csv.gz',
 'tmdb_api_results_2001.json',
 'title_basics.csv.gz',
 ' final_tmdb_data_2001.csv.gz',
 '.ipynb_checkpoints',
 'title_akas.csv.gz',
 'tmdb_results_combined.csv.gz',
 'title_ratings.csv.gz']

In [2]:
# Load in TMDB API cedentials
with open('/Users/jamesjones/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [3]:
# Assign API key
tmdb.API_KEY = login['api-key']

In [4]:
# Define a our function to get movie with the rating (certification) included
imdb_id = tmdb.Movies('imdb_id')

def get_movie_with_rating(imdb_id):
    # Get the movie object for the current id
    movie = tmdb.Movies(imdb_id)

# Save the .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()

# Loop through countries in releases
    for c in releases['countries']:
        if c['iso_3166_1'] == 'US':
        # Save a "certificatio" key in the info dictionary 
            info['certification'] = c['certification']
    return info

# How we built this out will be explained below

In [5]:
# Define function to write new .json files

def write_json(new_data, filename): 
    # Appends a list of records (new_data) to a json file (filename)
     #Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/
    
    with open(filename, 'r+') as file:
        
        # First we load existing data into a dict.
        file_data = json.load(file)
        
        # Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
                
        # Sets file's current position at offset.
        file.seek(0)
        
        # Convert back to json.
        json.dump(file_data, file)

In [6]:
# Make a movie object using .Movies function (explained in tmdb)
movie = tmdb.Movies(603)
    # This is a sample run to find where the info our stakeholder wants is stored

In [7]:
# Movie objects have a .info() dictionary
info = movie.info()
info

{'adult': False,
 'backdrop_path': '/l4QHerTSbMI7qgvasqxP36pqjN6.jpg',
 'belongs_to_collection': {'id': 2344,
  'name': 'The Matrix Collection',
  'poster_path': '/bV9qTVHTVf0gkW0j7p7M0ILD4pG.jpg',
  'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'},
 'budget': 63000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.warnerbros.com/matrix',
 'id': 603,
 'imdb_id': 'tt0133093',
 'original_language': 'en',
 'original_title': 'The Matrix',
 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
 'popularity': 64.865,
 'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg',
 'production_companies': [{'id': 79,
   'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
   'name': 'Village Roadshow Pictures',
   'origin_country': 'US'},
  {'id': 174,
   'logo_path': '/IuAlhI9eVC9Z8UQWOIDdWRKSEJ.png'

In [8]:
# We know our stakeholder is looking for Revenue, Budget, and certification (rating)
    # Certification is not shown in the above output (we'll come back to it)
print(info['budget'])
print(info['revenue'])

# We will also view the imdb_id, which we will need later
print(info['imdb_id'])

63000000
463517383
tt0133093


In [9]:
# We can, using our API readme, search our certification
 # Let's try an example
movie = tmdb.Movies('tt0133093')
response = movie.releases()
response

{'id': 603,
 'countries': [{'certification': '',
   'iso_3166_1': 'NO',
   'primary': False,
   'release_date': '1999-07-07'},
  {'certification': 'M/12',
   'iso_3166_1': 'PT',
   'primary': False,
   'release_date': '1999-06-09'},
  {'certification': 'R',
   'iso_3166_1': 'US',
   'primary': False,
   'release_date': '1999-03-24'},
  {'certification': '',
   'iso_3166_1': 'FI',
   'primary': False,
   'release_date': '1999-06-05'},
  {'certification': '',
   'iso_3166_1': 'TH',
   'primary': False,
   'release_date': '2021-09-03'},
  {'certification': '15',
   'iso_3166_1': 'GB',
   'primary': False,
   'release_date': '1999-06-11'},
  {'certification': 'PG-12',
   'iso_3166_1': 'JP',
   'primary': False,
   'release_date': '1999-09-11'},
  {'certification': '12',
   'iso_3166_1': 'NL',
   'primary': False,
   'release_date': '1999-06-17'},
  {'certification': 'R',
   'iso_3166_1': 'US',
   'primary': False,
   'release_date': '1999-03-30'},
  {'certification': 'K-16',
   'iso_3166_1

### You can see above that we rating, but there are MANY because there are many countries
- Our stakeholder is ONLY interested in US movies, so we need to select them

In [10]:
# Here is an example
response = movie.releases()
for c in movie.countries:
    if c['iso_3166_1'] == 'US':
        print(c['certification'])

R
R


Now, we know that "info" contains the movie 'revenue' and 'budget', and "releases" contains our 'certification'. We therefore need to pull from both to obtain these 3 pieces of information that our stakeholder is looking for. Let's put it all together

In [11]:
# Get the movie object for the current id
movie = tmdb.Movies('tt0133093')

# Save the .info .releases dictionaries
info = movie.info()
releases = movie.releases()

# Loop through countries in releases
for c in releases['countries']:
    if c['iso_3166_1'] == 'US':
        # Save a "certificatio" key in the info dictionary 
        info['certification'] = c['certification']
        
# *This is adapted into our get_movie_with_rating_funtion at the begining*

In [12]:
# Do a check, and see that certification has been included in the bottom!
 # Remember: info was set to tmdb.Movies(603) which included the imdb_id of 'tt0133093' used above
info

{'adult': False,
 'backdrop_path': '/l4QHerTSbMI7qgvasqxP36pqjN6.jpg',
 'belongs_to_collection': {'id': 2344,
  'name': 'The Matrix Collection',
  'poster_path': '/bV9qTVHTVf0gkW0j7p7M0ILD4pG.jpg',
  'backdrop_path': '/bRm2DEgUiYciDw3myHuYFInD7la.jpg'},
 'budget': 63000000,
 'genres': [{'id': 28, 'name': 'Action'},
  {'id': 878, 'name': 'Science Fiction'}],
 'homepage': 'http://www.warnerbros.com/matrix',
 'id': 603,
 'imdb_id': 'tt0133093',
 'original_language': 'en',
 'original_title': 'The Matrix',
 'overview': 'Set in the 22nd century, The Matrix tells the story of a computer hacker who joins a group of underground insurgents fighting the vast and powerful computers who now rule the earth.',
 'popularity': 64.865,
 'poster_path': '/f89U3ADr1oiB1s9GkdPOEpXUk5H.jpg',
 'production_companies': [{'id': 79,
   'logo_path': '/tpFpsqbleCzEE2p5EgvUq6ozfCA.png',
   'name': 'Village Roadshow Pictures',
   'origin_country': 'US'},
  {'id': 174,
   'logo_path': '/IuAlhI9eVC9Z8UQWOIDdWRKSEJ.png'

### Our stakeholder is only interested in movies that came out in 2000, 2001, so we need to filter for those  

In [13]:
YEARS_TO_GET = [2000, 2001]

In [14]:
# Load in Title Basics data created in Part 1 (per our instructions)
basics = pd.read_csv('Data/title_basics.csv.gz')
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El Tango del Viudo y Su Espejo Deformante,0,2020.0,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,126,Drama


In [15]:
# Create a for loop to filter (loop) through years
    # OUTER LOOP
for YEARS in tqdm_notebook(YEARS_TO_GET, desc = 'YEARS', position = 0):

# Define the JSON file to store results for year
    # This will show at the beginng 
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEARS}.json'
    
    # Check to see if our file exists
    file_exists = os.path.isfile(JSON_FILE)
    
    # Now, create the file if it exists, or leave it if it does exist
    if file_exists == False:
    
        # Save an empty dict to a new json file to append later
        with open(JSON_FILE, 'w') as f:
            json.dump([{'imdb_id':0}],f)
        
    # Now, save our specifisc years as the current df YEAR = 2000
    df_2000 = basics.loc[basics['startYear'] == YEARS].copy()
    df_2000.head(3)

    # Save movie ids from the year 2000 to a list
    movie_ids = df_2000['tconst'].copy()
    movie_ids

    # Check for any previous data
    previous_df = pd.read_json(JSON_FILE)
    previous_df # Will be empty at first
    
    # Filter out any ids that already exist in our JSON_FILE (helps when returning to our problem later)
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
# Get index and movie id from list
    # INNER loop

    for movie_id in tqdm_notebook(movie_ids_to_get,
                                      desc = f'Movies from {YEARS}',
                                      position = 1,
                                      leave = True):
            # Make attempt to retrieve data for movie id
        try:
            temp = get_movie_with_rating(movie_id) # Our first function

                # Extend results of our file with our other function
            write_json(temp, JSON_FILE)

                # Create a short sleep (helps bog down the server less)
            time.sleep(0.02)

            # If it fails, make a dict with just the id
        except Exception as e:
            continue
            
        # Finally, store our info in the JSON_FILE
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER} final_tmdb_data_{YEARS}.csv.gz",
                            compression = 'gzip', index = False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/208 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/241 [00:00<?, ?it/s]

In [19]:
# Print the number of errors that the loop encountered
    # Aka, number of movie ids that caused an issue
print(f"- Total errors: {len(Exception)}")

TypeError: object of type 'type' has no len()

In [17]:
final_year_df

,imdb_id,adult,backdrop_path,belongs_to_collection,budget,genres,homepage,id,original_language,original_title,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,certification
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,tt0035423,0.0,/hfeiSfWYujh6MKhtGTXyK3DD4nN.jpg,None,48000000.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 14, ...",,11232.0,en,Kate & Leopold,...,76019048.0,118.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"If they lived in the same century, they'd be p...",Kate & Leopold,0.0,6.324,1155.0,PG-13
2,tt0114447,0.0,None,None,0.0,"[{'id': 53, 'name': 'Thriller'}, {'id': 28, 'n...",,151007.0,en,The Silent Force,...,0.0,90.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,They left him for dead... They should have fin...,The Silent Force,0.0,5.000,3.0,NaN
3,tt0118589,0.0,/9NZAirJahVilTiDNCHLFcdkwkiy.jpg,None,22000000.0,"[{'id': 18, 'name': 'Drama'}, {'id': 10402, 'n...",,10696.0,en,Glitter,...,5271666.0,104.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,"In music she found her dream, her love, herself.",Glitter,0.0,4.600,118.0,PG-13
4,tt0118652,0.0,/mWxJEFRMvkG4UItYJkRDMgWQ08Y.jpg,None,1000000.0,"[{'id': 27, 'name': 'Horror'}, {'id': 9648, 'n...",,17140.0,en,The Attic Expeditions,...,0.0,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,His search for peace of mind... will leave his...,The Attic Expeditions,0.0,5.100,28.0,R
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1290,tt7797790,0.0,None,None,0.0,"[{'id': 27, 'name': 'Horror'}]",,956219.0,en,Edmund Kemper Part 3: La mort sévit,...,0.0,72.0,"[{'english_name': 'French', 'iso_639_1': 'fr',...",Released,,Edmund Kemper Part 3: La mort sévit,0.0,0.000,0.0,NaN
1291,tt8665056,0.0,None,None,0.0,"[{'id': 37, 'name': 'Western'}]",http://skeletoncreekproductions.com/p-movie-br...,885436.0,en,Guns Along The Bravo,...,0.0,85.0,[],Released,Evil came to the Southwest until three blazing...,Guns Along The Bravo,0.0,0.000,0.0,
1292,tt8795764,0.0,None,None,0.0,"[{'id': 27, 'name': 'Horror'}]",https://www.utahwolf.com/films/coming-soon-new...,871624.0,en,New Breed,...,0.0,57.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Released,,New Breed,0.0,0.000,0.0,NR
1293,tt9071078,0.0,None,None,0.0,"[{'id': 28, 'name': 'Action'}]",http://www.hkcinemagic.com/en/movie.asp?id=6627,201706.0,cn,致命密函,...,0.0,90.0,"[{'english_name': 'Cantonese', 'iso_639_1': 'c...",Released,,Chinese Heroes,0.0,3.000,2.0,NaN
